# **DL-5. Fine-tuning & Transfer Learning**

# 1. Введение

В этом модуле речь пойдёт о **Переносе обучения**, или **Transfer learning**.

Обучение нейронной сети требует большого объёма данных, например, в ImageNet это миллионы картинок. Что делать, если в текущей задаче не так много данных? Оказывается, в решении новой задачи можно переиспользовать часть нейросети, которая училась на *ImageNet*.

*ImageNet* — база данных аннотированных изображений, предназначенная для отработки и тестирования методов распознавания образов и машинного зрения. ImageNet использует краудсорсинг для аннотирования изображений. ImageNet — это dataset, организованный в соответствии с иерархией WordNet.

Сеть, которая обучилась на ImageNet, рассматривают, как состоящую из:

* первая часть (на рисунке синяя) состоит из **конволюционных слоёв** и извлекает из картинки признаки;
* вторая часть (на рисунке оранжевая) с помощью **многослойного персептрона (MLP)** по признакам определяет, к какому из тысячи классов принадлежит картинка.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/2715587cc14f6ef255902a2a02e5234d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_1_1.png)

Первую (синюю) часть такой сети можно переиспользовать в новой задаче. Такой подход называется **Переносом обучения**, или **Transfer learning**. Таким образом, чтобы обучить сеть на новой задаче, необходимо доучить только крайний полносвязный слой.

Нейроны на разных слоях сети, которую учили на ImageNet, реагируют по-разному:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/3c10a7700a7df1157e47447b94f4675d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_2.png)

## Дополнительные материалы

В качестве дополнительной литературы рекомендуем вам:

* прочесть статью «[Feature Visualization. How neural networks build up their understanding of images](https://distill.pub/2017/feature-visualization/)»;
* ознакомиться с видео-материалом «[AvitoNet: сервис компьютерного зрения в Avito](https://www.youtube.com/watch?v=c7d2E_zkju8&feature=youtu.be)» .

# 2. Как получить такие картинки

Как получить картинки, которые рисуют, на что реагируют нейроны?

Нейронная сеть дифференцируема, каждый нейрон является сложной функцией. Достаточно подобрать вход, который даст максимальную активацию на отдельно взятый нейрон. Аргументами функции, которые подбираются с помощью обратного распространения ошибки, является входное изображение. Фактически организуется стохастический градиентный спуск, в ходе которого изображение приобретает осмысленный вид.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/f308d77db72f4d6609970be2bf61f4fb/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_3.png)

Строить такие картинки можно, используя целый слой или канал в слое. Задача: найти, на что среагирует нейрон, при помощи **метода градиентного спуска (SGD)**.

Если взять на картинке один нейрон, то картинка, которая будет подобрана при помощи SGD, будет содержать кусочек текстуры только в той части изображения, на которую смотрит этот нейрон. Каждый нейрон имеет ограниченное поле обзора. «Слепых пятен» можно избежать, взяв все нейроны слоя. 

Можно максимизировать выход одного нейрона, канала, слоя или целого выхода сети для конкретного класса.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0e0ef5c2a7db67ab769a88a41a5d9eb6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_4.png)

Также можно взять выход нейронной сети, который соответствует конкретному классу, и под него подбирать изображение при помощи SGD.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/5a748b96a65ad7101abcd79a8d9a91ca/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_6.png)

Другой способ: в dataset, используемом при обучении, взять картинки, на которых конкретный **нейрон даёт максимальную активацию**.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/da64bf36b24163f8feed60cf0cb8d88d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_5.png)

В этом случае нужно намного меньше данных для обучения последних слоёв.

Перенос обучения работает, если новая задача похожа на ImageNet (содержит те же объекты или похожие). Стоит также учесть, что в ImageNet нет лиц.

# 3. Перенос обучения

Разберём задачу распознавания эмоций по фотографии. Если бы нейронная сеть обучалась нами с нуля, то экстрактор выглядел бы следующим образом:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0cfbbccf0402adbf58de873b1dc4da5b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_7.png)

В ImageNet-сети, наверняка, уже есть детекторы краёв и простые текстуры, которые можно использовать в нашей задаче. Остальные текстуры необходимо будет выучить, используя дополнительные данные для дообучения сети.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/57eb8d0f6e1e6e2562061c2962212104/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_8.png)

Взяв первые несколько слоёв из фича-экстрактора, который обучен по ImageNet, останется дообучить только последние слои нейронной сети.

# 4. Fine-tuning

Рассмотрим сеть, которая обучена на ImageNet. Фиксируем в этой сети часть фича-экстрактора (зелёная на рисунке).

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/1832924a1ad3c7541a148cf8e972452a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_9.png)

Тогда в новой задаче дообучить предстоит только синий и тёмно-зелёный слои.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/626ab1f6165cf83867a8d90d1a0e7c1a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_10.png)

Здесь используется **Fine-tuning** — связный приём, очень похожий на Transfer learning. Заключается он в следующем. Тёмно-зелёный слой зафиксируем, а синий слой будем инициализировать не случайными значениями, а теми весами, которые получили от ImageNet-сети. Далее пустим градиенты с маленьким шагом, чтобы подстроить слои под нашу задачу.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0e8a094ccba91f5a7add1ce07458baee/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_11.png)

## Преимущества Fine-tuning

Fine-tuning — это очень популярный подход, так как:

* ImageNet покрывает огромное количество объектов, которые есть в реальном мире.
* В Keras есть веса для обученных на ImageNet VGG, Inception, ResNet и так далее.
* Можно легко сделать ансамбль из этих сетей.

**Резюмируя**, обратимся к следующей таблице:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/62e7fe5a6a6d57172a78f08f7271f615/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_12.png)

## Дополнительные материалы

В качестве дополнительной литературы рекомендуем вам прочесть статью «[A Comprehensive guide to Fine-tuning Deep Learning Models in Keras (Part I)](https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html)»

# 5. Автокодировщики

**Автокодировщики (autoencoders)** — это специальная архитектура нейронной сети, позволяющая сжимать входные данные. Автокодировщик сжимает входные признаки в более компактное представление, которое не теряет информации об исходных данных.

Encoder сжимает признаковое описание на входе, decoder расшифровывает эту информацию обратно, не изменяя входные данные:

* *Encoder = data to hidden;*
* *Decoder = hidden to data;*
* *Decoder(Encoder(x)) ~ x (близость по MSE).*

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/89c36578717e43afd768667f2c8d6e94/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_13.png)

Такое сжатое представление, или **bottleneck** (горлышко бутылки), будет полезным для решения задач.

## Вспомним про PCA (метод главных компонент)

Данный метод основан на представлении исходной матрицы признаков в виде произведения других матриц и позволяет найти пространство меньшей размерности, хорошо описывающее входные данные. Этот метод в следующем модуле будет разобран подробно.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/540f9d625d9a3bbbfb4d92cdc69a13e5/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_14.png)

Такую задачу можно переписать в виде нейронной сети:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/f220660b107b879e59eca47b1f156ba0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_15.png)

**D признаков** на входе при помощи полносвязного слоя **Encoder** сжимаем до представления в **H признаков**.

H намного меньше D.

Полносвязный **слой Decoder** с помощью маленького представления H пытается восстановить исходные D признаков.

**Цель** обучения такой нейросети в том, чтобы представление H, пропущенное через сеть, не изменяло входные данные.


Метод главных компонент позволяет сжимать картинку в десять раз.

# 6. Un-pooling

Результат предыдущего метода можно улучшить, ведь для изображений применяются свёрточные нейронные сети.

**Добавим свёрточные слои**. К исходной картинке применим свёрточный слой, описывая каждый пиксель изображения по пикселям, которые были в окрестности исходного изображения. Добавим ещё несколько слоёв.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/affe00c658d66f42775e0d3492fdac80/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_16.png)

На выходе предстоит «схлопнуть» представление в картинку исходного расширения с одним или тремя (в случае цветного изображения) выходными каналами. При помощи среднеквадратичных потерь можно обучить это преобразование. Но это достаточно наивный подход, так как требует слишком много вычислений и не содержит "bottleneck".

Чтобы получить компактное представление в таких сетях, добавим **pooling-слой**.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/2163a299c1fbbfc8328f79f88cd4b0ce/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_17.png)

Pooling-слой будет сжимать изображение в два раза. На выходе получим небольшой тензор, который можно вытянуть в вектор, это и будет представлением для картинки. Это представление будем называть embedding,  результатом encoder'а.

Как развернуть полученный вектор из числа обратно в картинку? Кажется естественным повторить преобразование а в обратную сторону. Как обратить результат pooling-слоя? Используем Un-pooling-слой.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/57bf29a92c2112c74ab0f3837e3b7dd9/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_18.png)

Un-pooling
Рассмотрим простой вариант. Будем делать un-pooling при помощи заполнения ячеек теми же значениями, которые были рядом, интерполируя значения ближайшего соседа.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0100fd27d2c0e7c8e17bc0f8c2089c4c/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_19.png)

Такой простой подход работает. Свёрточные слои после un-pooling сгладят «плохие» преобразования. Всю эту сеть можно обучить при помощи средней квадратичной ошибки MSE.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/48c980b8d3a7e7b0e5d526bdb07ffdea/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_20.png)

# 7. Поиск похожих изображений

Пусть нам нужно сравнить два изображения.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/953a1c114b87687dd80156a231c2b8a8/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_21.png)

Понятно, что на картинках может быть один и тот же объект, но с разных ракурсов. Сравнивать попиксельно невозможно — необходимы признаки более высокого уровня, которые может дать конволюционная сеть.

Возьмём большое количество таких лиц и прогоним через **autoencoder**. Результат работы encoder'a (bottleneck) для каждого изображения запомним как вектор. Поиск похожих изображений будет заключаться в поиске похожих векторов. И это работает!

**Denoising autoencoders** — ещё одно применение autoencoders. Denoising autoencoders учатся удалять шум с изображения.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/78e3032d6d6f94e9ce3f0b36d269b362/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_22.png)

# 8. Быстрый KNN

Метод **aproximate k-nearest neighbors** позволяет приблизительно найти k ближайших соседей.

[Методы](https://github.com/erikbern/ann-benchmarks), работающие по этому принципу, устроены более или менее одинаково.  Они пытаются специальным образом проиндексировать пространство embedding'ов, чтобы в нём можно было быстро искать вектор, который наиболее похож на заданный.

Например, на задачах **Fashion MNIST** можно взять вектор, который получается из нашей нейронной сети, и по нему искать ближайшие картинки. На представленном графике по горизонтальной оси отложена полнота поиска, по вертикальной — сколько запросов в секунду мы можем выполнять для такого поиска.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/d69a97e8e8b67b1644b61764bc51b9cd/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_23.png)

## Полнота поиска

Пусть по выбранной картинке честно найдены 100 штук наиболее похожих на неё. Приближённым способом также ищем 100 наиболее похожих на нашу картинку. 

Если в сотне приближённо найденных картинок найдётся 95 из сотни реально похожих, будем говорить, что полнота равна 95 %. В этом случае можно сказать, что приближённый метод отработал хорошо.

Самые продвинутые методы основаны на **hnsw** и дают 10 000 запросов в секунду при полноте в 95 %.

## Дополнительные материалы
	
В качестве дополнительной литературы рекомендуем вам прочесть:

* «[Benchmarking nearest neighbors](https://github.com/erikbern/ann-benchmarks)»
* «[Code for reproducing 2nd place solution for Kaggle competition IEEE's Signal Processing Society](https://github.com/ikibardin/kaggle-camera-model-identification)»

# 9. Пространство представлений

Рассмотрим на примере, как работает поиск похожих изображений.

Возьмём картинку, на которой человек улыбается, превратим её в вектор и найдём ближайшие картинки по близости векторов. Окажется, что на ближайших картинках изображены лица людей, которые имеют схожий ракурс и улыбку.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/8e3cc60683327a37f830930c7dc00aa1/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_24.png)

Другой пример: для не улыбающегося лица в очках найдутся следующие похожие картинки:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/53478e19f3251d639b43da075b33a95d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_25.png)

В пространстве embedding'ов мы можем делать интерполяцию, плавно меняя картинку:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/9fbb6de35a1bd0cc6ff5aa6df4a39fb6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_26.png)

# 10. Вывод

Для чего нужны autoencoder'ы?

* Нам не нужны размеченные данные! Учим экстрактор признаков за бесплатно!
* Полученный экстрактор можно файн-тьюнить под нашу задачу!

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0da151704d0e580d9c51e7cec5d989c7/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DL-2_5_%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_27.png)

В этом модуле мы рассмотрели два важных подхода к нейросетевым представлениям:

1. **Fine-tuning / Transfer learning**, в котором смотрим на часть нейросети как на экстрактор признаков для нашей задачи, который можем полностью или частично переиспользовать.
2. Второй подход заключается в использовании неразмеченных данных, где учим autoencoder и его encoder-часть берём как экстрактор признаков.

# 11. Практика. Часть 1

Ноутбук к скринкасту на [kaggle](https://www.kaggle.com/itslek/transfer-learning-keras-flowers-sf-dl-v1?scriptVersionId=31741245).

# 12. Практика. Часть 2

Ноутбук к скринкасту на [kaggle](https://www.kaggle.com/itslek/transfer-learning-keras-flowers-sf-dl-v1?scriptVersionId=31741245). Обратите внимание, что ноутбук был обновлён для совместимости с TF2.

# 13. Практика. Часть 3

Ноутбук к скринкасту на [kaggle](https://www.kaggle.com/itslek/transfer-learning-keras-flowers-sf-dl-v1?scriptVersionId=31741245).

# 14. Практика. Часть 4

Ноутбук к скринкасту на [kaggle](https://www.kaggle.com/itslek/transfer-learning-keras-flowers-sf-dl-v1?scriptVersionId=31741245).

# 15. Практика. Часть 5

Ноутбук к скринкасту на [kaggle](https://www.kaggle.com/itslek/transfer-learning-keras-flowers-sf-dl-v1?scriptVersionId=31741245).

Полезные ссылки:
* [Paperswithcode](https://paperswithcode.com/sota)
* [sotabench.com](https://sotabench.com/benchmarks/image-classification-on-imagenet)
* [efficientnet на GitHub](https://github.com/qubvel/efficientnet)